# PSyKE's demo for regression tasks

Some imports.

In [1]:
from psyke import Extractor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from psyke.utils.logic import pretty_theory
from sklearn.model_selection import train_test_split
import pandas as pd

Import an artificial dataset.

In [2]:
dataset = pd.read_csv("test/resources/datasets/arti.csv")

Split between train and test set in a reproducible way.

In [3]:
train, test = train_test_split(dataset, test_size = 0.3, random_state = 10)

We use as predictor a MLP and we train it.

In [6]:
predictor = Sequential(
    [
        Dense(25, kernel_initializer = 'uniform', activation = "tanh"),
        Dense(5, kernel_initializer = 'uniform', activation = "tanh"),
        Dense(1, kernel_initializer = 'uniform')
    ]

)

predictor.compile(loss = "mae", metrics = ['mae', 'mse'], optimizer = Adam(learning_rate = 0.1))

callbacks = [
    EarlyStopping(
            monitor = 'val_loss', 
            min_delta = .005, 
            patience = 30, 
            restore_best_weights = True, 
            verbose = 0
        )
]

_ = predictor.fit(train.iloc[:, :-1], train.iloc[:, -1], validation_split = 0.2,  
              epochs = 100, batch_size = 150, verbose = 0, callbacks = callbacks)

print((abs(predictor.predict(test.iloc[:, :-1]).flatten() - test.iloc[:, -1].values)).mean())

0.027154218554496772


We create an extractor that uses the REAL algorithm and we extract prolog rules from our trained MLP.

In [7]:
it = Extractor.iter(predictor, 1.0 / 30, 1, 600, 100, 0.2, False)
theory_from_iter = it.extract(train)
print('ITER extracted rules:\n\n' + pretty_theory(theory_from_iter))

ITER extracted rules:

z(X, Y, 0.4089) :-
    X in [0.0, 0.5164], Y in [0.4714, 1.0].
z(X, Y, 0.6632) :-
    X in [0.0, 0.5164], Y in [0.0, 0.4714].
z(X, Y, 0.3574) :-
    X in [0.5164, 1.0], Y in [0.0, 0.4714].
z(X, Y, 0.068) :-
    X in [0.5164, 1.0], Y in [0.4714, 1.0].
